In [ ]:
import tensorflow as tf
!pip install fasttext livelossplot googletrans Rouge keras_tuner talos bayesian-optimization scikit-optimize


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

Running on TPU  ['10.97.125.26:8470']


In [ ]:
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import seaborn as sns

import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, SpatialDropout1D, Reshape,Softmax, Reshape, Activation, Dropout, Input, LSTM, Embedding,GRU, AdditiveAttention, Flatten,Bidirectional, TimeDistributed,RepeatVector,Permute,Concatenate,Multiply,BatchNormalization,Dot, LayerNormalization
from tensorflow.keras.models import load_model, Model,Sequential, model_from_json
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from random import seed , choice
import re
import pandas as pd

import time

from sklearn.model_selection import train_test_split
from IPython.display import clear_output

import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
from nltk import word_tokenize,pos_tag
from nltk.stem.wordnet import WordNetLemmatizer
from collections import defaultdict        
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
from pathlib import Path

import fasttext.util
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
import pickle
from livelossplot.inputs.tf_keras import PlotLossesCallback
from googletrans import Translator
translator = Translator(service_urls=['translate.googleapis.com'])
from sklearn.neighbors import KNeighborsClassifier, NearestNeighbors
from scipy import spatial
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
import time
import copy
from google.colab import drive
import keras_tuner as kt
import talos as ta
from bayes_opt import BayesianOptimization, UtilityFunction
from skopt.space import Integer, Categorical, Real

drive.mount('/content/drive')



In [ ]:
PROCESS_DATA = False
SHOW_DATASET_INFO = True
PROCESS_TRAIN_DS = True
GET_WORD_DICT = False 
TRAIN_MODEL = True
RUN_NAIVE_APPROACH = False
units=1024#256
lr = 0.001 #0.0005
batch_size = 128#128 best until now
AUTO = tf.data.experimental.AUTOTUNE
batch_size = batch_size * tpu_strategy.num_replicas_in_sync

emb_dim = 300
ds_len = 1000000
ds_train_size = 1000000
trainset_percentage = 0.97
training_epochs = 2000

#np.random.seed(0)
#tf.random.set_seed(0)

In [ ]:
def get_embeddings_fb(ft,vocab):
    embeddings_matrix = np.zeros((len(vocab)+1,emb_dim))
    for word in vocab:
        embeddings_matrix[vocab[word],:] = [float(i) for i in ft.get_word_vector(word)]
            
    return embeddings_matrix

def get_naive_embeddings(vocab,tokenizer_lang,vocab_len,fname ):
    embeddings_matrix = np.zeros((vocab_len,emb_dim))
    f= open(fname, encoding="utf8")
    word_found = 0
    for line in f:
        values = line.split()
        if(values[0] in vocab):
            word = values[0]
            
            try:
                coefs = [float(i) for i in values[1:]]
                #word = np.expand_dims(word, -1)[0]
                word_index = tokenizer_lang[word]

                embeddings_matrix[word_index,:] = coefs

                if word_found <10:
                    print(word," Found / Index :",word_index)
                    word_found +=1
            except:
                print("word : ",values[0])
                continue
    f.close()
    
    return embeddings_matrix

## Create dataset, Analyse and saved it

In [ ]:

Emoji_Dict ={
    ":p" : "<>" ,
    ":)" :"<SMILE_SIMPLE>",
    ":(":"<SAD>",
    ":'(":"<CRY>",
    ":')":"<BLINK_EYE>",
    ":d" : "<BIG_SMILE>", 
    ":-)":"<SMILE_HAPPY>",
    ";-)":"<SMILE_A_LOT>",
    ":<})":"<MOUSTACHE_SMILE>",
    ":-||":"<MAD>",
    ":-(":"<SAD>", 
    ":'-(":"<CRY>",
    ":-))":"<HAPPY>",
    ":-*": "<KISS>",
    ":-P~":"<LICK>", 
    ":-o" :"<SURPRISED>",
    ":-|":"<GRIM>",
    ":-/":"<PERPLEXED>",
    "=:O"  :"<FRIGHTNED>",
    "<3": "<LOVE>",
    "*****": "<5_STARS>",
    "****": "<4_STARS>",
    "***": "<3_STARS>",
    "**": "<2_STARS>",
    "*": "<1_STARS>",
    ".org" : "<>"
    }

translationTable = str.maketrans("áéíóúàèìòùâêîôûãõç", "aeiouaeiouaeiouaoc")
string_check= re.compile('[^a-zA-Z.,?!\'<>]')#0-9

In [ ]:
language_model = fasttext.load_model('/content/drive/My Drive/Colab Notebooks/Translator/lid.176.ftz')


In [ ]:


def get_sentence_separated(text, naive_approach=False):
    text_review = []

    text = (text.replace("?"," ? ").replace("!"," ! ").replace("."," . "))
    text = text.replace("back to text", " ").replace(" s.", " saint").replace(" s.a.", " sa").replace("monterosso farmhouses", " ").replace(" prof.", " prof")
    
    return [ "<SOS> " +text+ " <EOS> " ]
    
def process_dataset(line):
   
    line = line.translate(translationTable)
    line = ' '.join(line.split('\t'))

    if line.find(".org") != -1:
        site = line.split(".org")[0].split()[-1]
        line = line.replace(site+".org","<URL>")
    elif line.find(".com") != -1:
        if len(line.split(".com"))> 0:
            try:
                site = line.split(".com")[0].split()[-1]
                line = line.replace(site+".com","<URL>")
            except:
                pass
    
    line = line.replace(" st."," saint")
    line = (re.sub('\.+', " . ", line).replace("<br />"," ")).replace("?"," ? ").replace(" ! "," ! ")

    for em in list(Emoji_Dict.keys()):
        line = line.replace(em,Emoji_Dict[em])

    stripped =[]
    tmp = nltk.word_tokenize(line)
    for w in tmp:
        #if  w.isdigit():
        #    stripped.append("<NUMBER>")
        #else:
        stripped.append(w)

    line = ' '.join(stripped)
    line = re.sub(string_check, ' ', (line)
                    .replace("-"," ")
                    .replace("´","'")
                    .replace("`","'")
                    .replace("'nt"," not")
                    .replace("i'm","i am")
                    .replace("i 'm","i am")
                    #.replace("'s"," ")
                    .replace("don't","do not")
                    .replace("can't","can not")
                    .replace("n't"," not")
                    .replace("'re"," are")
                    .replace("'d"," would")
                    .replace("'ve"," have")
                    .replace("'ll"," will")
                    .replace("'till"," until")
                    .replace("''"," ")
                    .replace(" b "," ")
                    .replace(" c "," ")
                    .replace(" d "," ")
                    .replace(" f "," ")
                    .replace(" g "," ")
                    .replace(" h "," ")
                    .replace(" j "," ")
                    .replace(" k "," ")
                    .replace(" l "," ")
                    .replace(" m "," ")
                    .replace(" n "," ")
                    .replace(" p "," ")
                    .replace(" q "," ")
                    .replace(" r "," ")
                    .replace(" s "," ")
                    .replace(" t "," ")
                    .replace(" v "," ")
                    .replace(" w "," ")
                    )
        
    return  ' '.join(line.split())

    
def read_dataset():
    
    english_list = []
    english_size = []

    portuguese_list = []
    portuguese_size = []

    portuguese_naive_list = []
    english_naive_list = []

    path = "/content/drive/My Drive/Colab Notebooks/Translator/en-pt.bicleaner07.txt"

    f=open(path,'r', encoding="utf8")
    Lines = f.readlines()
    Lines = Lines[0:ds_len]
    ds_size = len(Lines)
    print("Progress show at every ",int(ds_size*0.05)," lines")

    for i, vec in enumerate(Lines):
        vec = str(vec).lower()
        vec = vec.split('\t')

        eng_line = vec[0]
        pt_line = vec[1]

        if i% int(ds_size*0.05) == 0 and i>0:
            print("Progress : ",ds_size,"/",i)
            break

        english_list.append(process_dataset(eng_line))
        portuguese_list.append(process_dataset(pt_line))
        
    f.close()

    english_list_review = []
    pt_list_review = []
    eng_expected_lang= []
    pt_expected_lang = []
    for i , sent in enumerate(zip(english_list, portuguese_list)):
        en_txt, pt_txt = sent

        eng_lang = language_model.predict(en_txt, k=1)[0][0]
        pt_lang = language_model.predict(pt_txt, k=1)[0][0]

        en = get_sentence_separated(en_txt, naive_approach=False)
        pt = get_sentence_separated(pt_txt, naive_approach=False)
        if len(en) == len(pt):
            for s in en:
                english_size.append(len(s.split()))
                english_list_review.append(s)
                english_naive_list.append(s.replace("."," ").replace("'"," "))
                eng_expected_lang.append(eng_lang)
            for s in pt:
                portuguese_size.append(len(s.split()))
                pt_list_review.append(s)
                s_naive = s.replace("."," ").replace("'"," ")
                portuguese_naive_list.append(s_naive)
                pt_expected_lang.append(pt_lang)

              

    cols = ['eng','pt','eng_sentence_size', 'pt_sentence_size']
    df_tmp= pd.DataFrame(columns=cols)

    df_tmp["eng"] =english_list_review
    df_tmp["pt"] =pt_list_review
    df_tmp["eng_expected_lang"] =eng_expected_lang
    df_tmp["pt_expected_lang"] =pt_expected_lang
    df_tmp["english_naive"] =english_naive_list
    df_tmp["portuguese_naive"] =portuguese_naive_list
    df_tmp["eng_sentence_size"] =english_size
    df_tmp["pt_sentence_size"] =portuguese_size

    return  df_tmp

In [ ]:
#https://www.tensorflow.org/api_docs/python/tf/data/Dataset

def describre_dataset(dataset):
    plt.figure()
    pd.Series(dataset.eng_sentence_size).hist()
    plt.title("Sentence Size for training dataset")

    plt.show()
    print(pd.Series(dataset.pt_sentence_size).describe())

from sklearn.model_selection import train_test_split
def create_trainingdev(eng_sequence, pt_sequence, df,maxlen, pt_max_len, en_max_len):
    
    y = pad_sequences(eng_sequence, padding='post', maxlen=en_max_len)
    x = pad_sequences(pt_sequence, padding='post', maxlen=pt_max_len)
    
    x_train, x_dev, y_train, y_dev = train_test_split(x, y, test_size=(1-trainset_percentage), random_state=0)
    print("x_in shape", x_train.shape)
    print("y_in shape", y_train.shape)

    """train_elements_index = np.random.choice(len(x), int(len(x)*trainset_percentage), replace=False) 
    print("train_elements_index length : ",len(train_elements_index))
    
    x_train = np.take(x, train_elements_index, 0)
    y_train = np.take(y, train_elements_index, 0)
    print("x_in shape", x_train.shape)
    print("y_in shape", y_train.shape)


    x_dev= np.delete(x, train_elements_index, 0)
    y_dev= np.delete(y, train_elements_index, 0)"""
    print("x_dev shape", x_dev.shape)
    print("y_test shape", y_dev.shape)

    return x_train, y_train, x_dev, y_dev 



In [ ]:
df = read_dataset()
df = df.drop_duplicates()
df

KeyboardInterrupt: ignored

# Check text languages

In [ ]:
# eng_expected_lang =  language_model.predict(df.eng.tolist(), k=1)[0]
# pt_expected_lang=  language_model.predict(df.pt.tolist(), k=1)[0]

# eng_expected_lang = [x[0] for x in eng_expected_lang]
# pt_expected_lang = [x[0] for x in pt_expected_lang]
# df["eng_expected_lang"] =  eng_expected_lang
# df["pt_expected_lang"] =  pt_expected_lang
# df


In [ ]:
df["eng_expected_lang"].unique(), df["pt_expected_lang"].unique()


In [ ]:
df["eng_expected_lang"].value_counts()

As there are many different languages in the data, let's fix this and include only sentences with portuguese or english based languages

In [ ]:
df[df.eng_expected_lang != "__label__en"]

In [ ]:
df["pt_expected_lang"].value_counts()

In [ ]:
df[df.eng_expected_lang != "__label__pt"]

In [ ]:
df = df[df.eng_expected_lang.isin(["__label__en","__label__uk"])]
df = df[df.pt_expected_lang.isin(["__label__pt","__label__br"])]
df

# Remove sentence outliers

In [ ]:
sns.boxplot(x=df['eng_sentence_size'])
df.eng_sentence_size.describe()

In [ ]:
sns.boxplot(x=df['pt_sentence_size'])
df.pt_sentence_size.describe()

Tukeys' rule approach to fix outliers, considering a factor of 1.5 (not extreme outliers, as we have a huge dataset)

In [ ]:
Q1_en = df.eng_sentence_size.quantile(0.25)
Q3_en = df.eng_sentence_size.quantile(0.75)
IQR_en = Q3_en - Q1_en
factor = 1.5
IQR_en, (Q1_en - factor * IQR_en),(Q3_en + factor * IQR_en), len(df[ df.eng_sentence_size <= (Q3_en + factor * IQR_en)]), len(df[ df.eng_sentence_size > (Q3_en + factor * IQR_en)])


In [ ]:
Q1_pt = df.pt_sentence_size.quantile(0.25)
Q3_pt = df.pt_sentence_size.quantile(0.75)
IQR_pt = Q3_pt - Q1_pt
IQR_pt, (Q1_pt - 1.5 * IQR_pt),(Q3_pt + 1.5 * IQR_pt), len(df[ df.pt_sentence_size <= (Q3_pt + 1.5 * IQR_pt)]), len(df[ df.pt_sentence_size > (Q3_pt + 1.5 * IQR_pt)])


In [ ]:
IQR_total = np.minimum((Q3_pt + 1.5 * IQR_pt),(Q3_en + 1.5 * IQR_en))
IQR_total 

In [ ]:
if SHOW_DATASET_INFO :
    describre_dataset(df)
    #IQR_total = 120 # REVIEW LATER
    df = df[df.eng_sentence_size<IQR_total]
    df = df[df.eng_sentence_size> 0]#6

    if ds_train_size < ds_len:
      df = df.sample(n=ds_train_size, random_state=1).reset_index().drop(["index"], axis  =1)
    describre_dataset(df)

In [ ]:
df.duplicated().any()

In [ ]:
df.isna().any()

## Create, save and read train dataset 

In [ ]:
if PROCESS_DATA :
    #df = read_dataset()
    
    eng_tokenizer = Tokenizer(filters='"#$%&()*+-/:;=@[\\]^_´`«»{|}~\t\n',oov_token='<oov>')# sequence_size = 500 ~= mean+ std sentence size
    eng_tokenizer.fit_on_texts(df.eng.values.tolist())

    pt_tokenizer = Tokenizer(filters='"#$%&()*+-/:;=@[\\]^_´`«»{|}~\t\n',oov_token='<oov>')# sequence_size = 500 ~= mean+ std sentence size
    pt_tokenizer.fit_on_texts(df.pt.values.tolist())

    with open('/content/drive/My Drive/Colab Notebooks/Translator/eng_tokenizer.pkl', 'wb') as f:
            pickle.dump(eng_tokenizer, f)
    
    with open('/content/drive/My Drive/Colab Notebooks/Translator/pt_tokenizer.pkl', 'wb') as f:
            pickle.dump(pt_tokenizer, f)

    df.to_pickle("/content/drive/My Drive/Colab Notebooks/Translator//df_init.pkl")  
else:

    df = pd.read_pickle("/content/drive/My Drive/Colab Notebooks/Translator/df_init.pkl")
    
    with open('/content/drive/My Drive/Colab Notebooks/Translator/eng_tokenizer.pkl', 'rb') as f:
        eng_tokenizer = pickle.load(f)

    with open('/content/drive/My Drive/Colab Notebooks/Translator/pt_tokenizer.pkl', 'rb') as f:
        pt_tokenizer = pickle.load(f)

# Check words frequency

In [ ]:
def check_word_frequency(tokenizer_temp):
    word_frequency = {}
    for i, (word, count) in enumerate(tokenizer_temp.word_counts.items()):
        word_frequency[word]=count
            
    return word_frequency

eng_frequency = check_word_frequency(eng_tokenizer)
pt_frequency = check_word_frequency(pt_tokenizer)

eng_frequency = {k: v for k, v in sorted(eng_frequency.items(), key=lambda item: item[1], reverse=True)}
pt_frequency = {k: v for k, v in sorted(pt_frequency.items(), key=lambda item: item[1],reverse=True)}

In [ ]:
eng_frequency

In [ ]:
pt_frequency

In [ ]:
eng_frequency_tmp = {k:v for (k,v) in eng_frequency.items() if v > 5000}
plt.bar(*zip(*eng_frequency_tmp.items()))
plt.show()

In [ ]:
pt_frequency_tmp = {k:v for (k,v) in pt_frequency.items() if v > 5000}
plt.bar(*zip(*pt_frequency_tmp.items()))
plt.show()

As seem, there are multiple words/characters with a really huge frequency, which can bias the model. But since we are translating them, I guess we can't just remove these stop words and special characters like , . the, etc...

In [ ]:
# plt.bar(*zip(*pt_frequency.items()))
# plt.show()

In [ ]:
eng_sequence = eng_tokenizer.texts_to_sequences(df.eng.tolist())
pt_sequence = pt_tokenizer.texts_to_sequences(df.pt.tolist())

Let's debug our sequence, to understand if we are setting the right english/portuguese pair as input and output 

In [ ]:
for en_sent,pt_sent,ptss,engss in zip(eng_sequence[10000:10100],pt_sequence[10000:10100],df.pt[10000:10100].tolist(),df.eng[10000:10100].tolist()):
  #for x in pt_sent:
    #print(x,"---->",pt_tokenizer.index_word[x])
  print("ORIGINAL : ",ptss,"--------->",engss)
  pts = ' '.join([pt_tokenizer.index_word[x] for x in pt_sent])
  ens = ' '.join([eng_tokenizer.index_word[x] for x in en_sent])
  print(pts,"--------->",ens)



# Getting Embeddings Matrices for our sequences, both Portuguese and English

In [ ]:
eng_vocab_size = len(pt_tokenizer.word_index) +1
pt_vocab_size = len(eng_tokenizer.word_index) +1

ft = fasttext.load_model('/content/drive/My Drive/Colab Notebooks/Translator/cc.en.300.bin')
fasttext.util.reduce_model(ft, emb_dim)
embeddings_eng_matrix = get_embeddings_fb(ft,eng_tokenizer.word_index)
del ft

ft = fasttext.load_model('/content/drive/My Drive/Colab Notebooks/Translator/cc.pt.300.bin')
fasttext.util.reduce_model(ft, emb_dim)
embeddings_pt_matrix = get_embeddings_fb(ft,pt_tokenizer.word_index)

sequence_size = np.maximum(df.pt_sentence_size.max(),df.eng_sentence_size.max())

embeddings_eng_matrix, embeddings_pt_matrix, sequence_size


# Word translation for Naive Approach

Setting Word dictionary containing every portuguese/english word pair translation. Naive Approach focus in words, not sentences. So, instead of having the sentence pair of translation, we need the word pair of translation. For this, we need to use the google translator api to translate each word

In [ ]:
# TO use in the Naive Approach
try:
  if GET_WORD_DICT:
      eng_vocab_list = list(eng_tokenizer.word_index.keys())
      eng_vocab_list.remove("<oov>")
      eng_vocab_list.remove(".")
      eng_vocab_list.remove("<eos>")
      eng_vocab_list.remove("<number>")
      
      pt_vocab_list = []
      google_batch = 1000
      for i in range(0, len(eng_vocab_list), google_batch):
          if (i+google_batch) <  len(eng_vocab_list):
              aux = eng_vocab_list[i:i+google_batch]
          else :
              aux = eng_vocab_list[i:]

          print(i,"/",len(eng_vocab_list))
          translations = translator.translate(aux, src='en', dest='pt')
          for t in translations:
              pt_vocab_list.append(t.text)



      words_dict = pd.DataFrame(columns=["eng_words","pt_words"])
      words_dict["eng_words"] = eng_vocab_list
      words_dict["pt_words"] = pt_vocab_list

      words_dict.pt_words =  words_dict.pt_words.apply(lambda line : re.sub(string_check, ' ',line.lower().translate(translationTable)))

      words_dict.loc[words_dict.eng_words=="th","pt_words"]="nd"
      words_dict.pt_words =  words_dict.pt_words.apply(lambda x : ' '.join(x.split()) if len(x.split()) > 1 else x.split()[0])

      words_dict.to_pickle("./words_dict.pkl")  

      pt_tokenizer_naive = pt_tokenizer
      pt_tokenizer_naive.fit_on_texts(words_dict.pt_words.tolist())

      with open('pt_tokenizer_naive.pkl', 'wb') as f:
          pickle.dump(pt_tokenizer_naive, f)

  else:

      words_dict = pd.read_pickle("/content/drive/My Drive/Colab Notebooks/Translator/words_dict.pkl")

      with open('/content/drive/My Drive/Colab Notebooks/pt_tokenizer_naive.pkl', 'rb') as f:
          pt_tokenizer_naive = pickle.load(f)
except:
  pass

# Naive Approach -  Word translation 

We need to have a matrix R which will transform X into Y 

## 2.1 Translation as linear transformation of embeddings


Given dictionaries of English and French word embeddings you will create a transformation matrix `R`
* Given an English word embedding, $\mathbf{e}$, you can multiply $\mathbf{eR}$ to get a new word embedding $\mathbf{f}$.
    * Both $\mathbf{e}$ and $\mathbf{f}$ are [row vectors](https://en.wikipedia.org/wiki/Row_and_column_vectors).
* You can then compute the nearest neighbors to `f` in the french embeddings and recommend the word that is most similar to the transformed word embedding.


### Describing translation as the minimization problem

Find a matrix `R` that minimizes the following equation. 

$$\arg \min _{\mathbf{R}}\| \mathbf{X R} - \mathbf{Y}\|_{F}\tag{1} $$

### Frobenius norm

The Frobenius norm of a matrix $A$ (assuming it is of dimension $m,n$) is defined as the square root of the sum of the absolute squares of its elements:

$$\|\mathbf{A}\|_{F} \equiv \sqrt{\sum_{i=1}^{m} \sum_{j=1}^{n}\left|a_{i j}\right|^{2}}\tag{2}$$


### Actual loss function
In the real world applications, the Frobenius norm loss:

$$\| \mathbf{XR} - \mathbf{Y}\|_{F}$$

is often replaced by it's squared value divided by $m$:

$$ \frac{1}{m} \|  \mathbf{X R} - \mathbf{Y} \|_{F}^{2}$$

where $m$ is the number of examples (rows in $\mathbf{X}$).

* The same R is found when using this loss function versus the original Frobenius norm.
* The reason for taking the square is that it's easier to compute the gradient of the squared Frobenius.
* The reason for dividing by $m$ is that we're more interested in the average loss per embedding than the  loss for the entire training set.
    * The loss for all training set increases with more words (training examples),
    so taking the average helps us to track the average loss regardless of the size of the training set.


### *********************************************************************************************************************************************

#### 1: Computing the loss
* The loss function will be squared Frobenoius norm of the difference between
matrix and its approximation, divided by the number of training examples $m$.
* Its formula is:
$$ L(X, Y, R)=\frac{1}{m}\sum_{i=1}^{m} \sum_{j=1}^{n}\left( a_{i j} \right)^{2}$$

where $a_{i j}$ is value in $i$th row and $j$th column of the matrix $\mathbf{XR}-\mathbf{Y}$.


### 2: Computing the gradient of loss in respect to transform matrix R

* Calculate the gradient of the loss with respect to transform matrix `R`.
* The gradient is a matrix that encodes how much a small change in `R`
affect the change in the loss function.
* The gradient gives us the direction in which we should decrease `R`
to minimize the loss.
* $m$ is the number of training examples (number of rows in $X$).
* The formula for the gradient of the loss function $𝐿(𝑋,𝑌,𝑅)$ is:

$$\frac{d}{dR}𝐿(𝑋,𝑌,𝑅)=\frac{d}{dR}\Big(\frac{1}{m}\| X R -Y\|_{F}^{2}\Big) = \frac{2}{m}X^{T} (X R - Y)$$

In [ ]:
def get_matrices(word_dict_df,eng_tokenizer,pt_tokenizer_naive,emb_dim, sequence_size, embeddings_eng_matrix_naive,embeddings_pt_matrix_naive):

    eng_embedding_layer = Embedding(input_dim = len(eng_tokenizer.word_index)+1,
        output_dim = emb_dim,
        input_length = sequence_size ,
        name ="eng_embedding_layer", 
        weights=[embeddings_eng_matrix_naive], 
        trainable=False, mask_zero=True
        )

    pt_embedding_layer = Embedding(input_dim = len(pt_tokenizer_naive.word_index)+1,
        output_dim = emb_dim,
        input_length = sequence_size ,
        name ="pt_embedding_layer", 
        weights=[embeddings_pt_matrix_naive], 
        trainable=False, mask_zero=True
        )
        
    X_l = list()
    Y_l = list()

    for en_word, pt_word in zip(word_dict_df.eng_words.tolist(), word_dict_df.pt_words.tolist()):

        if len(pt_word.split())>1 :
            pt_vec = [0]*emb_dim
            try : 
                for  w in pt_word.split():
                    pt_vec += pt_embedding_layer(pt_tokenizer_naive.word_index[w])
                #pt_vec = pt_vec/
            except:
                print(f"Failed to get complete sentence'{pt_word}'" )
                continue
        else :
            try :
                pt_vec = pt_embedding_layer(pt_tokenizer_naive.word_index[pt_word])
            except:
                print(f"Failed to get complete '{pt_word}'" )
                continue

        en_vec = eng_embedding_layer(eng_tokenizer.word_index[en_word])

        X_l.append(en_vec)
        Y_l.append(pt_vec)

    X_train = np.vstack(X_l)
    Y_train = np.vstack(Y_l)

    X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.2, random_state=42)

    return eng_embedding_layer, pt_embedding_layer, X_train, Y_train, X_val, Y_val

def compute_loss(X, Y, R):
    '''
    Inputs: 
        X: a matrix of dimension (m,n) where the columns are the English embeddings.
        Y: a matrix of dimension (m,n) where the columns correspong to the French embeddings.
        R: a matrix of dimension (n,n) - transformation matrix from English to French vector space embeddings.
    Outputs:
        L: a matrix of dimension (m,n) - the value of the loss function for given X, Y and R.
    '''
    m = len(X)
    diff = np.dot(X,R) - Y
    diff_squared = diff**2
    sum_diff_squared = np.sum(diff_squared)
    loss = sum_diff_squared/m
    return loss

def compute_gradient(X, Y, R):
    '''
    Inputs: 
        X: a matrix of dimension (m,n) where the columns are the English embeddings.
        Y: a matrix of dimension (m,n) where the columns correspong to the French embeddings.
        R: a matrix of dimension (n,n) - transformation matrix from English to French vector space embeddings.
    Outputs:
        g: a scalar value - gradient of the loss function L for given X, Y and R.
    '''
    m = len(X)
    gradient = (2/m)*np.dot((X.T), (np.dot(X,R)-Y))
    return gradient


def align_embeddings(X, Y, train_steps=100, learning_rate=0.0003, verbose=True, compute_loss=compute_loss, compute_gradient=compute_gradient):
    '''
    Inputs:
        X: a matrix of dimension (m,n) where the columns are the English embeddings.
        Y: a matrix of dimension (m,n) where the columns correspong to the French embeddings.
        train_steps: positive int - describes how many steps will gradient descent algorithm do.
        learning_rate: positive float - describes how big steps will  gradient descent algorithm do.
    Outputs:
        R: a matrix of dimension (n,n) - the projection matrix that minimizes the F norm ||X R -Y||^2
    '''
    np.random.seed(129)

    # the number of columns in X is the number of dimensions for a word vector (e.g. 300)
    # R is a square matrix with length equal to the number of dimensions in th  word embedding
    R = np.random.rand(X.shape[1], X.shape[1])

    for i in range(train_steps):
        if verbose and i % 1000 == 0:
            print(f"loss at iteration {i} is: {compute_loss(X, Y, R):.4f}")
        gradient = compute_gradient(X, Y, R)
        # update R by subtracting the learning rate times gradient
        R -= learning_rate * gradient

    return R


def nearest_neighbor_adapted(v, candidates, k=1):
    """
    Input:
      - v, the vector you are going find the nearest neighbor for
      - candidates: a set of vectors where we will find the neighbors
      - k: top k nearest neighbors to find
    Output:
      - k_idx: the indices of the top k closest vectors in sorted form
    """
    ### START CODE HERE ###
    similarity_l = []

    # for each candidate vector...
    v_array = v.reshape((1,candidates.shape[1]))

    cos_similarity = cosine_similarity(v_array, candidates)
    similarity_l = list(cos_similarity[0,:])
    
    # sort the similarity list and get the indices of the sorted list    
    sorted_ids = np.argsort(similarity_l)  
    # Reverse the order of the sorted_ids array
    sorted_ids = np.argsort(similarity_l)[::-1]
    # get the indices of the k most similar candidate vectors
    k_idx = sorted_ids[:k]
    ### END CODE HERE ###
    return k_idx

    
def test_vocabulary(X, Y, R, nearest_neighbor=nearest_neighbor_adapted):
    '''
    Input:
        X: a matrix where the columns are the English embeddings.
        Y: a matrix where the columns correspong to the French embeddings.
        R: the transform matrix which translates word embeddings from
        English to French word vector space.
    Output:
        accuracy: for the English to French capitals
    '''

    pred = np.dot(X,R)

    # initialize the number correct to zero
    num_correct = 0

    # loop through each row in pred (each transformed embedding)
    for i in range(len(pred)):
        # get the index of the nearest neighbor of pred at row 'i'; also pass in the candidates in Y
        pred_idx = nearest_neighbor(pred[i],Y)[0]
        # if the index of the nearest neighbor equals the row of i... \
        if pred_idx == i:
            # increment the number correct by 1.
            num_correct += 1

    # accuracy is the number correct divided by the number of rows in 'pred' (also number of rows in X)
    accuracy = num_correct/len(pred)

    ### END CODE HERE ###

    return accuracy

In [ ]:
RUN_NAIVE_APPROACH = False
if RUN_NAIVE_APPROACH:

    embeddings_eng_matrix_naive =  get_embeddings_fb(ft,eng_tokenizer.word_index)
    embeddings_pt_matrix_naive = get_embeddings_fb(ft,pt_tokenizer_naive.word_index)
    eng_embedding_layer, pt_embedding_layer,  X_train, Y_train, X_val, Y_val = get_matrices(words_dict, eng_tokenizer,pt_tokenizer_naive,emb_dim, sequence_size, embeddings_eng_matrix_naive, embeddings_pt_matrix_naive)


In [ ]:
if RUN_NAIVE_APPROACH:
    #Calculate transformation matrix R
    R_train = align_embeddings(X_train, Y_train, train_steps=200000, learning_rate=0.01) #1982.9182


loss at iteration 0 is: 32.9974

loss at iteration 1000 is: 14.2506

loss at iteration 2000 is: 9.6578

loss at iteration 3000 is: 7.4478

loss at iteration 4000 is: 6.0210

loss at iteration 5000 is: 4.9834

loss at iteration 6000 is: 4.1927

loss at iteration 7000 is: 3.5761

loss at iteration 8000 is: 3.0881

loss at iteration 9000 is: 2.6975

loss at iteration 10000 is: 2.3817

loss at iteration 11000 is: 2.1240

loss at iteration 12000 is: 1.9121

loss at iteration 13000 is: 1.7363

loss at iteration 14000 is: 1.5894

loss at iteration 15000 is: 1.4658

loss at iteration 16000 is: 1.3610

loss at iteration 17000 is: 1.2717

loss at iteration 18000 is: 1.1949

loss at iteration 19000 is: 1.1286

loss at iteration 20000 is: 1.0709

loss at iteration 21000 is: 1.0205

loss at iteration 22000 is: 0.9762

loss at iteration 23000 is: 0.9370

loss at iteration 24000 is: 0.9022

loss at iteration 25000 is: 0.8712

loss at iteration 26000 is: 0.8434

loss at iteration 27000 is: 0.8184

loss at iteration 28000 is: 0.7958

loss at iteration 29000 is: 0.7753

loss at iteration 30000 is: 0.7567

loss at iteration 31000 is: 0.7397

loss at iteration 32000 is: 0.7241

loss at iteration 33000 is: 0.7098

loss at iteration 34000 is: 0.6967

loss at iteration 35000 is: 0.6846

loss at iteration 36000 is: 0.6734

loss at iteration 37000 is: 0.6630

loss at iteration 38000 is: 0.6534

loss at iteration 39000 is: 0.6445

loss at iteration 40000 is: 0.6361

loss at iteration 41000 is: 0.6284

loss at iteration 42000 is: 0.6211

loss at iteration 43000 is: 0.6144

loss at iteration 44000 is: 0.6081

loss at iteration 45000 is: 0.6021

loss at iteration 46000 is: 0.5966

loss at iteration 47000 is: 0.5914

loss at iteration 48000 is: 0.5865

loss at iteration 49000 is: 0.5819

loss at iteration 50000 is: 0.5775

loss at iteration 51000 is: 0.5734

loss at iteration 52000 is: 0.5696

loss at iteration 53000 is: 0.5660

loss at iteration 54000 is: 0.5626

...
loss at iteration 172000 is: 0.4993

loss at iteration 173000 is: 0.4993

In [ ]:
if RUN_NAIVE_APPROACH:
    acc = test_vocabulary(X_val, Y_val, R_train)  # this might take a minute or two
    print(f"accuracy on test set is {acc:.3f}")

In [ ]:
if RUN_NAIVE_APPROACH:
    v = embeddings_eng_matrix_naive[eng_tokenizer.word_index["you"]]
    candidates = embeddings_eng_matrix_naive
    indices = nearest_neighbor_adapted(v, candidates, 5)
    print(indices,"-->", [pt_tokenizer_naive.index_word[x] for x in indices])#, "-->",candidates[indices])

In [ ]:
if RUN_NAIVE_APPROACH:
    #Calculate transformation matrix R
    R_train = align_embeddings(X_train[:500], Y_train[:500], train_steps=10000, learning_rate=0.1) #1982.9182

    acc = test_vocabulary(X_val[:300], Y_val[:300], R_train)  # this might take a minute or two
    print(f"accuracy on test set is {acc:.3f}")

# Deep Learning Approach

Create the dataset for deep learning approach, as long os the data parameters such as decoder and encoder size, training size, etc..

In [ ]:

pt_vocab_size = len(pt_tokenizer.word_index) +1
eng_vocab_size = len(eng_tokenizer.word_index) +1

sequence_size = np.maximum(df.pt_sentence_size.max(),df.eng_sentence_size.max())
pt_sequence_size = df.pt_sentence_size.max() # sequence_size
en_sequence_size = df.eng_sentence_size.max() #sequence_size 
eng_sequence = eng_tokenizer.texts_to_sequences(df.eng.tolist())
pt_sequence = pt_tokenizer.texts_to_sequences(df.pt.tolist())
x, y, x_dev, y_dev, = create_trainingdev(eng_sequence,pt_sequence, df,sequence_size,pt_sequence_size,en_sequence_size)


In [ ]:
dataset_fit = tf.data.Dataset.from_tensor_slices((x,y))
dataset_fit = dataset_fit.shuffle(buffer_size = 10000).batch(batch_size=batch_size, drop_remainder=True)
dataset_fit_dev = tf.data.Dataset.from_tensor_slices((x_dev,y_dev))
dataset_fit_dev = dataset_fit_dev.shuffle(buffer_size = 10000).batch(batch_size=batch_size, drop_remainder=True)

dataset_fit_hp = tf.data.Dataset.from_tensor_slices((x,y))
dataset_fit_hp =dataset_fit_hp.shuffle(buffer_size = 10000).batch(batch_size=batch_size, drop_remainder=True)
dataset_fit_dev_hp = tf.data.Dataset.from_tensor_slices((x_dev,y_dev))
dataset_fit_dev_hp = dataset_fit_dev_hp.shuffle(buffer_size = 10000).batch(batch_size=batch_size, drop_remainder=True)

train_size = len(list(dataset_fit))

dataset_fit = dataset_fit.repeat()
dataset_fit_dev = dataset_fit_dev.repeat()

del ft

In [ ]:
decoder_vocab_size = len(eng_tokenizer.word_index) +1
encoder_vocab_size = len(pt_tokenizer.word_index) +1

Double check the translation pairs

In [ ]:

for xi,yi in dataset_fit.take(5):
  print(' '.join([pt_tokenizer.index_word[i] for i in xi[0].numpy() if i != 0]))
  print(' '.join([eng_tokenizer.index_word[i] for i in yi[0].numpy() if i != 0]), "\n ------------------------------- \n\n")

for xi_batch, yi_batch in dataset_fit_dev.take(1):
    print(' '.join([pt_tokenizer.index_word[i] for i in xi[0].numpy() if i != 0]))
    print(' '.join([eng_tokenizer.index_word[i] for i in yi[0].numpy() if i != 0]), "\n ------------------------------- \n\n")

### Defining the Encoder, Decoder and Attention classes for models

In [ ]:
import matplotlib.ticker as ticker

def plot_attention(attention, sentence, predicted_sentence,pltsize=(10, 10)):
  sentence = sentence.split()
  predicted_sentence = predicted_sentence.split()
  fig = plt.figure(figsize=pltsize)
  ax = fig.add_subplot(1, 1, 1)

  attention = attention[:len(predicted_sentence), :len(sentence)]

  ax.matshow(attention, cmap='viridis', vmin=0.0)

  fontdict = {'fontsize': 14}

  ax.set_xticklabels([''] + sentence, fontdict=fontdict, rotation=90)
  ax.set_yticklabels([''] + predicted_sentence, fontdict=fontdict)

  ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
  ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

  ax.set_xlabel('Input text')
  ax.set_ylabel('Output text')
  plt.suptitle('Attention weights')

  
class EncoderTranslator(tf.keras.Model):
    def __init__(self, vocab_size, emb_dim,units, enc_embedding_layer, drop = 0.5, reg = 0.01):  
        super(EncoderTranslator, self).__init__()
        self.emb_dim = emb_dim
        self.enc_vocab_size = vocab_size
        self.enc_embedding_layer = enc_embedding_layer
        self.enc_units = units

        self.embedding = Embedding(input_dim=self.enc_vocab_size, input_length = pt_sequence_size, output_dim=self.emb_dim, weights=[self.enc_embedding_layer], trainable=False, mask_zero=True )
        self.lstm = LSTM(self.enc_units, return_sequences=True, return_state=True, dropout=drop, recurrent_dropout=drop, recurrent_regularizer = tf.keras.regularizers.L1(l1=reg))
        # self.spatial = SpatialDropout1D(0.5)

    def call(self, inp, state=None):
        x = self.embedding(inp)
        enc_mask = self.embedding.compute_mask(inp)

        state = self.lstm.get_initial_state(x)

        output, h_state, c_state = self.lstm(x, initial_state=state)
        #output = self.spatial(output)
        
        return output, (h_state, c_state), enc_mask


class SelfMaskedDotProductAttention(tf.keras.Model): 
    def __init__(self):
        super(SelfMaskedDotProductAttention, self).__init__()


    def call(self, q, k, v, padding_mask= None, look_ahead_mask= None, q_mask= None):
        
        PRINT_SHAPE= False
                
        matmul_qk = tf.matmul(q, k, transpose_b=True) 

        dk = tf.cast(tf.shape(k)[-1], tf.float32)
        scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)
        if PRINT_SHAPE : print("weight shape : ", scaled_attention_logits.shape," - [batch_size, Tq, Tv]")
        
        if padding_mask is not None:
            padding_mask = tf.expand_dims(padding_mask, 1)
            scaled_attention_logits = tf.where(padding_mask, scaled_attention_logits, tf.experimental.numpy.full_like(scaled_attention_logits,  -1e9))

        # if q_mask is not None:
        #     q_mask = tf.expand_dims(q_mask, 2)
        #     scaled_attention_logits = tf.where(q_mask, scaled_attention_logits, tf.experimental.numpy.full_like(scaled_attention_logits,  -1e9))

        if look_ahead_mask is not None:
            scaled_attention_logits += (look_ahead_mask * -1e9)
            

        attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1) 
        if PRINT_SHAPE : print("attention_weights / values : ",attention_weights.shape, v.shape)

        context_vector = tf.matmul(attention_weights, v)

        if PRINT_SHAPE : print("context_vector shape : ", context_vector.shape)

        return context_vector, attention_weights


class DecoderTranslator(tf.keras.Model):
    def __init__(self, vocab_size, emb_dim, units,  embeddings_dec_layer, causal_mask_enabled= True, use_attention = False, drop = 0.5, reg = 0.01):
        super(DecoderTranslator, self).__init__()

        self.use_attention = use_attention
        self.causal_mask_enabled = causal_mask_enabled
        self.dec_vocab_size = vocab_size
        self.emb_dim = emb_dim
        self.embeddings_dec_layer = embeddings_dec_layer
        self.units= units

        self.embedding = Embedding(input_dim=self.dec_vocab_size, input_length = en_sequence_size, output_dim=self.emb_dim,  weights=[self.embeddings_dec_layer], trainable=False  , mask_zero=True)#embedding_dim #
        #self.spatial = SpatialDropout1D(drop_spat)
        self.lstm = LSTM(self.units,return_sequences=True, return_state=True, dropout=drop, recurrent_dropout=drop, recurrent_regularizer = tf.keras.regularizers.L1(l1=reg))
        self.drop = Dropout(drop)
        # self.batchNorm0 =BatchNormalization()
        # self.layernorm0 = LayerNormalization(epsilon=layernorm_epsi)
        #self.layernorm1 = LayerNormalization(epsilon=layernorm_epsi)
        self.d0 = Dense(self.units, activation='relu')
        # self.drop1 = Dropout(0.5)
        # self.batchNorm1 =BatchNormalization()
        # self.d1 = Dense(self.units, activation='relu')
        self.d = Dense(self.dec_vocab_size)
        self.dot_attention = SelfMaskedDotProductAttention()
        

    def create_look_ahead_mask(self, size, size_input):
        mask = 1 - tf.linalg.band_part(tf.ones((size, size_input)), -1, 0)
        return mask  

    def call(self, decoder_input, dec_hidden,  enc_output,  inp_mask= None, target_mask= None, training= True):

        #target_mask = self.embedding.compute_mask(decoder_input)

        x = self.embedding(decoder_input)
        #x = self.spatial(x)
        x, state_h, state_c = self.lstm(x, initial_state = dec_hidden )

        if self.causal_mask_enabled :
            look_ahead_mask = self.create_look_ahead_mask(tf.shape(decoder_input)[1], tf.shape(enc_output)[1])
        else:
            look_ahead_mask = None

        if self.use_attention:
            #tf.expand_dims(dec_hidden[0], axis = 1)
            context_vector, attention_weights = self.dot_attention(x, enc_output, enc_output, padding_mask = inp_mask, look_ahead_mask =look_ahead_mask, q_mask = target_mask)#dec_hidden[0]
            x =  tf.concat([context_vector, x], axis=-1)
            #x = tf.reduce_sum(context_vector, axis = 1)
        else:
            context_vector = None
            attention_weights = None

        #lstm_out, state_h, state_c = self.lstm(x, initial_state = dec_hidden )
        #x = lstm_out
        x = self.drop(x)#, training = training
        # x = self.layernorm0(x)
        x = self.d0(x)
        #x = self.layernorm1(x)
        # x = self.drop1(x)
        # x = self.batchNorm1(x)
        # x = self.d1(x)

        output = self.d(x)

        return output, (state_h, state_c), attention_weights



#### Check the classes structures

* Encoder

In [ ]:
for i, o in dataset_fit.take(1):
    print(i)

i_batch = i
i = i[0:1]
o = o[0:1]
encoder = EncoderTranslator(encoder_vocab_size, emb_dim, units, embeddings_pt_matrix)
enc_output, enc_state, mask_enc = encoder(i)
mask = (i != 0) 

print("mask : ",mask)
print("mask_enc : ",mask_enc)
print(f'Input batch, shape (batch): {i.shape}')
print(f'Output batch, shape (batch): {o.shape}')

print(f'Encoder output, shape (batch, s, units): {enc_output.shape}')
print(f'Encoder hidden state, shape (batch, units): {enc_state[0].shape}')
print(f'Encoder cell state, shape (batch, units): {enc_state[0].shape}')

* Decoder

In [ ]:
decoder = DecoderTranslator(decoder_vocab_size, emb_dim, units, embeddings_eng_matrix, causal_mask_enabled= True)
qmask = (o != 0) 
dec_output, dec_state, attention_weights = decoder(o, enc_state, enc_output,  inp_mask = mask,  target_mask = qmask)


print(f'Target batch, shape (batch): {o.shape}')
print(f'Target batch tokens, shape (batch, s): {yi.shape}')
print(f'Decoder output, shape (batch, s, units): {dec_output.shape}')
print(f'Decoder hidden state, shape (batch, units): {dec_state[0].shape}')
print(f'Decoder cell state, shape (batch, units): {dec_state[1].shape}')

* Masked Dot Product Attention - This one was chosed because there are no major layers to be learned on it, which implies simplicity, and it has been used for transformers. 

In [ ]:
#@title Default title text
attention_layer = SelfMaskedDotProductAttention()

print("i -->",i[0])

query= enc_output
qmask = (i != 0) 
values = enc_output

print("Query shape : ",query.shape)
print("Values shape : ", values.shape)
attention_context, attention_weights_pad_mask = attention_layer(query,  values, values, padding_mask = mask)
attention_context, attention_weights_with_qmask = attention_layer(query,  values, values, q_mask = qmask)
attention_context, attention_weights_nomask = attention_layer(query,  values, values)
attention_context, attention_weights_with_causal= attention_layer(query,  values, values, look_ahead_mask= (1 - tf.linalg.band_part(tf.ones((i.shape[1], i.shape[1])), -1, 0)) )
attention_context, attention_weights = attention_layer(query,  values, values, look_ahead_mask= (1 - tf.linalg.band_part(tf.ones((i.shape[1], i.shape[1])), -1, 0)) , padding_mask = mask, q_mask = qmask)
attention_context, attention_weights_with_qmask_padding = attention_layer(query,  values, values, padding_mask = mask, q_mask = qmask)

print("Attention result shape: (batch size, s, units) {}".format(attention_context.shape))
print("Attention weights shape: (batch_size, sequence_length, sequence_length) {}".format(attention_weights.shape))

In [ ]:

max_limit = np.count_nonzero((i!= 0), axis=1).max()
plt.figure(figsize=(30,20))
plt.subplot(3, 3, 1)
plt.imshow(attention_weights_nomask[0][ :max_limit,:max_limit])#[ :max_limit,:max_limit]
plt.title('Attention weights - No mask')

plt.subplot(3, 3, 2)
plt.imshow(attention_weights_pad_mask[0][ :max_limit,:max_limit])#[ :max_limit,:max_limit]
plt.title('Attention weights - Padding mask')

plt.subplot(3, 3, 3)
plt.imshow(attention_weights_with_qmask[0][ :max_limit,:max_limit])#[ :max_limit,:max_limit]
plt.title('Attention weights with query mask')

plt.subplot(3, 3, 4)
plt.imshow(attention_weights_with_qmask_padding[0][ :max_limit,:max_limit])#[ :max_limit,:max_limit]
plt.title('Attention weights with query and padding mask')

plt.subplot(3, 3, 5)
plt.imshow(attention_weights_with_causal[0][ :max_limit,:max_limit])#[ :max_limit,:max_limit]
plt.title('Attention weights with causal mask')

plt.subplot(3, 3, 6)
plt.imshow(attention_weights[0][ :max_limit,:max_limit])#[ :max_limit,:max_limit]
plt.title('Attention weights with all masks')


plt.subplot(3, 3, 7)
plt.pcolormesh(i_batch != 0)
plt.title('Mask')

* Translator model class creation

In [ ]:

class MaskedLossCustom(tf.keras.losses.Loss):
  def __init__(self):
    self.name = 'masked_loss_transformer_custom'
    self.loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

  def __call__(self, real, pred):
      
    loss = self.loss_object(real, pred)

    # Mask off the losses on padding.
    mask = tf.cast(real != 0, tf.float32)
    loss *= mask

    # Return the total.
    return tf.reduce_sum(loss)/tf.reduce_sum(mask) 
    
# class MaskedLossCustom(tf.keras.losses.Loss):
#   def __init__(self):
#     self.name = 'masked_loss_transformer_custom'
#     self.loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

#   def __call__(self, real, pred):
      
#     mask = tf.math.logical_not(tf.math.equal(real, 0))
#     loss_ = self.loss_object(real, pred)

#     mask = tf.cast(mask, dtype=loss_.dtype)
#     loss_ *= mask

#     return tf.reduce_sum(loss_)/tf.reduce_sum(mask) 

  
class TrainCustomTranslator(tf.keras.Model):
    def __init__(self, emb_dim, units,encoder_vocab_size,decoder_vocab_size,embeddings_dec_matrix,embeddings_enc_matrix,dec_tokenizer, dec_sequence_size, enc_sequence_size, use_tf_function=True,
                 enc_drop = 0.5, enc_reg = 1e-5,  dec_drop = 0.5, dec_reg = 0.01, causal_mask_enabled= False, use_attention = False):
        super().__init__()

         
        self.encoder = EncoderTranslator(encoder_vocab_size, emb_dim, units, embeddings_enc_matrix, drop = enc_drop, reg = enc_reg)
        self.decoder = DecoderTranslator(decoder_vocab_size, emb_dim, units, embeddings_dec_matrix, causal_mask_enabled= causal_mask_enabled, use_attention = use_attention, drop = dec_drop, reg = dec_reg)
        self.decoder_vocab_size = decoder_vocab_size
        self.dec_tokenizer = dec_tokenizer
        self.use_tf_function = use_tf_function
        self.enc_sequence_size = enc_sequence_size
        self.dec_sequence_size = dec_sequence_size

  
    def accuracy_function(self, real, pred):
        real = tf.cast(real, dtype=tf.int32)
        accuracies = tf.equal(real, tf.cast(tf.argmax(pred, axis=2), dtype=tf.int32))

        mask = tf.math.logical_not(tf.math.equal(real, 0))
        accuracies = tf.math.logical_and(mask, accuracies)

        accuracies = tf.cast(accuracies, dtype=tf.float32)
        mask = tf.cast(mask, dtype=tf.float32)
        return tf.reduce_sum(accuracies)/tf.reduce_sum(mask)

    def call(self, inputs):
      inp, targ = inputs  
      input_mask = inp != 0

      enc_output, dec_hidden  = self.encoder(inp, None)
      last_dec= None
      dec_input =targ[:, :-1] ## Teacher forcing - feeding the target as the next input
      target_mask = None#dec_input != 0

      predictions, dec_hidden, context_vector, last_dec = self.decoder(dec_input,  dec_hidden, enc_output, last_dec, input_mask, target_mask)
      
      return predictions

    def train_step(self, inputs):
        if self.use_tf_function:
            return self._tf_train_step(inputs)
        else:
            return self._train_step(inputs)

    def _train_step(self, inputs):
        inp, targ = inputs  
        
        # input_mask = inp != 0

        with tf.GradientTape() as tape:
            
            enc_output, dec_hidden, input_mask  = self.encoder(inp, None)

            acc = []
            last_dec= None
            loss = tf.constant(0.0)

            # Pass enc_output to the decoder
            dec_input =targ[:, :-1] ## Teacher forcing - feeding the target as the next input
            # target_mask = dec_input != 0

            predictions, dec_hidden, context_vector = self.decoder(dec_input,  dec_hidden, enc_output,  input_mask)
            tar = targ[:, 1:]
            loss = self.loss(tar, predictions)

            self.compiled_metrics.update_state(tar, predictions)
            acc.append(self.metrics[0].result())
            average_acc  = tf.reduce_sum(tf.convert_to_tensor(acc) )
            acc_m = self.accuracy_function(tar, predictions)

        variables = self.trainable_variables 
        gradients = tape.gradient(loss, variables)
        self.optimizer.apply_gradients(zip(gradients, variables))

        m = {m.name: average_acc for m in self.metrics}
        m["loss"] = loss
        m["masked_acc"] = acc_m

        return m


    # def _train_step(self, inputs):
    #     inp, targ = inputs  
        
    #     input_mask = inp != 0
    #     target_mask = targ != 0
    #     max_target_length = tf.shape(targ)[1]

    #     with tf.GradientTape() as tape:
            
    #         enc_output, dec_hidden  = self.encoder(inp, None)
    #         dec_state = dec_hidden
            
    #         loss = tf.constant(0.0)
    #         acc = tf.constant(0.0)
    #         acc_m = tf.constant(0.0)
    #         predictions_list = []
    #         res = []

    #         for t in tf.range(max_target_length-1):

    #           new_tokens = targ[:, t:t+2]
    #           input_token, target_token = new_tokens[:, 0:1], new_tokens[:, 1:2]

    #           predictions, dec_state, context_vector = self.decoder(input_token,  dec_state, enc_output, input_mask, target_mask)

    #           y = target_token
    #           y_pred = predictions

    #           loss += self.loss(y, y_pred)
          
    #           self.compiled_metrics.update_state(target_token, predictions)
    #           acc += self.metrics[0].result()

    #           predictions_list.append(y_pred)
    #           res.append(y)
            
    #         acc_m = self.accuracy_function(tf.convert_to_tensor(res), tf.convert_to_tensor(predictions_list))
    #         average_loss = loss / tf.reduce_sum(tf.cast(target_mask, tf.float32))
    #         average_acc = acc / tf.reduce_sum(tf.cast(target_mask, tf.float32))


    #     variables = self.trainable_variables 
    #     gradients = tape.gradient(loss, variables)
    #     self.optimizer.apply_gradients(zip(gradients, variables))

    #     m = {m.name: average_acc for m in self.metrics}
    #     m["loss"] = average_loss
    #     m["masked_acc"] = acc_m

    #     return m

    @tf.function(input_signature=[[tf.TensorSpec(dtype=tf.int32, shape=[None, pt_sequence_size]),tf.TensorSpec(dtype=tf.int32, shape=[None, en_sequence_size])]])
    def _tf_train_step(self, inputs):
        return self._train_step(inputs)


    def test_step(self, inputs):
        return self._test_step(inputs)

    def _test_step(self, inputs):
        inp, targ = inputs   
        input_mask = inp != 0

        enc_output, dec_hidden, input_mask  = self.encoder(inp, None)

        acc = []
        last_dec= None
        loss = tf.constant(0.0)

        # Pass enc_output to the decoder
        dec_input =targ[:, :-1] ## Teacher forcing - feeding the target as the next input
        # target_mask = dec_input != 0
        
        predictions, dec_hidden, context_vector = self.decoder(dec_input,  dec_hidden, enc_output, input_mask, training= False)

        tar = targ[:, 1:]
        loss += self.loss(tar, predictions)

        self.compiled_metrics.update_state(tar, predictions)
        acc.append(self.metrics[0].result())
        average_acc  = tf.reduce_mean(tf.convert_to_tensor(acc) )

        m = {m.name: average_acc for m in self.metrics}#.update({'loss': batch_loss})
        m["loss"] = loss
        m["masked_acc"] = self.accuracy_function(tar, predictions)

        return m

    # def _test_step(self, inputs):
    #     inp, targ = inputs  
        
    #     input_mask = inp != 0
    #     target_mask = targ != 0
    #     max_target_length = tf.shape(targ)[1]
    
    #     enc_output, dec_hidden  = self.encoder(inp, None)
    #     dec_state = dec_hidden

    #     loss = tf.constant(0.0)
    #     acc = tf.constant(0.0)
    #     acc_m = tf.constant(0.0)

    #     predictions_list = []
    #     res = []
    #     for t in tf.range(max_target_length-1):

    #       new_tokens = targ[:, t:t+2]
    #       input_token, target_token = new_tokens[:, 0:1], new_tokens[:, 1:2]
          
    #       predictions, dec_state, context_vector = self.decoder(input_token,  dec_state, enc_output, input_mask, target_mask, training= False)

    #       y = target_token
    #       y_pred = predictions

    #       loss += self.loss(y, y_pred)
      
    #       self.compiled_metrics.update_state(target_token, predictions)
    #       acc += self.metrics[0].result()

    #       predictions_list.append(y_pred)
    #       res.append(y)
            
    #     acc_m = self.accuracy_function(tf.convert_to_tensor(res), tf.convert_to_tensor(predictions_list))
    #     average_loss = loss / tf.reduce_sum(tf.cast(target_mask, tf.float32))
    #     average_acc = acc / tf.reduce_sum(tf.cast(target_mask, tf.float32))


    #     m = {m.name: average_acc for m in self.metrics}
    #     m["loss"] = average_loss
    #     m["masked_acc"] = acc_m

    #     return m

    # @tf.function(input_signature=[[tf.TensorSpec(dtype=tf.int32, shape=[None, pt_sequence_size]),
    #                             tf.TensorSpec(dtype=tf.int32, shape=[None, en_sequence_size])
    #                             ]])
    # def _tf_test_step(self, inputs):
    #     return self._test_step(inputs)



In [ ]:
# with tpu_strategy.scope():

#   custom_translator = TrainCustomTranslator(
#         emb_dim= emb_dim,
#         units = units,
#         encoder_vocab_size= encoder_vocab_size,
#         decoder_vocab_size= decoder_vocab_size,
#         embeddings_dec_matrix = embeddings_eng_matrix,
#         embeddings_enc_matrix = embeddings_pt_matrix,
#         dec_tokenizer = eng_tokenizer,
#         dec_sequence_size = en_sequence_size, 
#         enc_sequence_size = pt_sequence_size,
#         use_tf_function=True,
#         causal_mask_enabled= True, use_attention = True)


#   # Configure the loss and optimizer
#   custom_translator.compile(
#       optimizer=tf.optimizers.Adam(0.001),
#       loss= MaskedLossCustom(),
#       metrics = [tf.metrics.SparseCategoricalAccuracy()] #["tf.metrics.SparseCategoricalAccuracy()"] AccuracyCustom()
#   )

#   history = custom_translator.fit(dataset_fit,  epochs=epochs, callbacks=[PlotLossesCallback()], steps_per_epoch = train_size,  shuffle= True,  validation_data = dataset_fit_dev, validation_steps=10)

# Performing Hyperparam tunning manual  + TPU

In [ ]:
import json
import pprint
pp = pprint.PrettyPrinter(indent=4)
MANUAL_TUNING = False
from datetime import datetime

def get_manual_tunning(train_ds, val_ds, hp_units, hp_enc_drop, hp_enc_reg, hp_dec_drop, hp_dec_reg,lr):
    tf.keras.backend.clear_session()

    with tpu_strategy.scope():
        custom_translator = TrainCustomTranslator(
          emb_dim= emb_dim,
          units = hp_units,
          encoder_vocab_size= encoder_vocab_size,
          decoder_vocab_size= decoder_vocab_size,
          embeddings_dec_matrix = embeddings_eng_matrix,
          embeddings_enc_matrix = embeddings_pt_matrix,
          dec_tokenizer = eng_tokenizer,
          dec_sequence_size = en_sequence_size, 
          enc_sequence_size = pt_sequence_size,
          use_tf_function=True,

          enc_drop =hp_enc_drop,
          enc_reg = hp_enc_reg, 
          dec_drop = hp_dec_drop, 
          dec_reg = hp_dec_reg,
          
          causal_mask_enabled= True, use_attention = True)

      
        custom_translator.compile(
          optimizer=tf.optimizers.Adam(lr),
          loss=MaskedLossCustom(),
          metrics = [tf.metrics.SparseCategoricalAccuracy()] )

        # Step 4: Train the model on TPU with fixed batch size.
        val_metric = 'val_masked_acc'
        stop_early = tf.keras.callbacks.EarlyStopping(monitor=val_metric, patience=100)
        out = custom_translator.fit(train_ds,  epochs=300,   shuffle= True, validation_data=val_ds, verbose =0, callbacks=[stop_early]) #callbacks=[PlotLossesCallback()],

    
    return out

if MANUAL_TUNING:

  L1L2_reg = [ 0.1, 0.01, 0.001, 0.0001]
  drop_reg = [ 0.1, 0.3, 0.5]

  para = {
      "hp_units": list(range(32,512,32)),
      "hp_enc_drop": drop_reg,
      "hp_enc_reg": L1L2_reg,
      "hp_dec_drop": drop_reg,
      "hp_dec_reg": L1L2_reg,
      "learning_rate": [  0.001, 0.0001]
      }

  Trial_Dict = {}
  trials = 0
  best_trial = -1
  best_trial_value = 0
  HP_tunning_summary = {}
  last_trial = -1
  LOAD_MANUAL_TUNNING = True

  if LOAD_MANUAL_TUNNING:
    try:
      with open("/content/drive/My Drive/Colab Notebooks/Translator/Trial_Dict.json") as fp:
        Trial_Dict= json.load(fp)
        print("Trial_Dict->",Trial_Dict)

      with open("/content/drive/My Drive/Colab Notebooks/Translator/HP_tunning_summary.json") as fp:
        HP_tunning_summary= json.load(fp)
      #print("HP_tunning_summary->",Trial_Dict)
      best_trial = HP_tunning_summary["Best_Trial"]
      best_trial_value = HP_tunning_summary["Best_Trial_Value"]
      last_trial = HP_tunning_summary["Trial_counter"]

      print(f"{best_trial} is the best trial with val_masked_acc = {best_trial_value} \n -----------------------------------\n")

    except Exception as e:
      print(f"Failed do read the json files . {e}")

  for hp_units in para["hp_units"]:
    for hp_enc_drop in para["hp_enc_drop"]:
        for hp_enc_reg in para["hp_enc_reg"]:
            for hp_dec_drop in para["hp_dec_drop"]:
                for hp_dec_reg in para["hp_dec_reg"]:
                  for learning_rate in para["learning_rate"]:

                    starting_point = datetime.now()

                    data = {
                      "hp_units": hp_units,
                      "hp_enc_drop": hp_enc_drop,
                      "hp_enc_reg": hp_enc_reg,
                      "hp_dec_drop": hp_dec_drop,
                      "hp_dec_reg": hp_dec_reg,
                      "learning_rate": learning_rate
                      }
                      
                    print(f"Trial {trials}\n Current architecture : ")
                    pp.pprint(data)
                    
                    if trials <= last_trial:
                      print(f"Current val_masked_acc : {Trial_Dict[str(trials)]['validation_score']} \n {best_trial} is the best trial with val_masked_acc = {best_trial_value} \n -----------------------------------\n")
                      trials = trials+1

                    else:

                      hist = get_manual_tunning(dataset_fit_hp ,dataset_fit_dev_hp,hp_units, hp_enc_drop,hp_enc_reg,hp_dec_drop,hp_dec_reg,learning_rate)

                      Trial_Dict[trials] = {
                          "data" : data,
                          "history" : hist.history,
                          "validation_score" : hist.history["val_masked_acc"][-1],
                      }
                      
                      if hist.history["val_masked_acc"][-1] > best_trial_value : 
                        best_trial_value = hist.history["val_masked_acc"][-1]
                        best_trial = trials

                      print(f"Current val_masked_acc : {hist.history['val_masked_acc'][-1]} \n {best_trial} is the best trial with val_masked_acc = {best_trial_value}")
                      
                      
                      duration = (datetime.now() - starting_point).total_seconds()

                      HP_tunning_summary = {
                          "Trials" : Trial_Dict,
                          "Best_Trial": best_trial,
                          "Best_Trial_Value" : best_trial_value,
                          "Trial_counter" : trials,
                          "Timer" : str(duration) +" seconds"

                      }
                      print(f"It took {str(duration)} seconds  \n -----------------------------------\n")

                      with open("/content/drive/My Drive/Colab Notebooks/Translator/Trial_Dict.json", "w") as fp:
                        json.dump(Trial_Dict,fp) 

                      with open("/content/drive/My Drive/Colab Notebooks/Translator/HP_tunning_summary.json", "w") as fp:
                        json.dump(HP_tunning_summary,fp) 

                      trials = trials+1

# Performing Hyperparam tunning with Bayesian Optimization

In [ ]:
#https://medium.com/analytics-vidhya/how-to-do-hyper-parameters-search-with-bayesian-optimization-for-keras-model-6b9918caa0b1
from bayes_opt.logger import JSONLogger
from bayes_opt.event import Events
from bayes_opt.util import load_logs

def get_bayesian_tunning(hp_units, hp_enc_drop, hp_dec_drop, hp_enc_reg, hp_dec_reg,lr):

    # tf.keras.backend.clear_session()
    with tpu_strategy.scope():
        hp_units = int(hp_units)
        custom_translator = TrainCustomTranslator(
          emb_dim= emb_dim,
          units = hp_units,
          encoder_vocab_size= encoder_vocab_size,
          decoder_vocab_size= decoder_vocab_size,
          embeddings_dec_matrix = embeddings_eng_matrix,
          embeddings_enc_matrix = embeddings_pt_matrix,
          dec_tokenizer = eng_tokenizer,
          dec_sequence_size = en_sequence_size, 
          enc_sequence_size = pt_sequence_size,
          use_tf_function=True,

          enc_drop = hp_enc_drop, enc_reg = hp_enc_reg,  dec_drop = hp_dec_drop, dec_reg = hp_dec_reg,
          causal_mask_enabled= True, use_attention = True)

        
        custom_translator.compile(
          optimizer=tf.optimizers.Adam(lr),
          loss=MaskedLossCustom(),
          metrics = [tf.metrics.SparseCategoricalAccuracy()] )

        # Step 4: Train the model on TPU with fixed batch size.
        val_metric = 'val_masked_acc'#val_sparse_categorical_accuracy
        stop_early = tf.keras.callbacks.EarlyStopping(monitor=val_metric, patience=100)
        out = custom_translator.fit(dataset_fit_hp ,  epochs=500,   shuffle= True, validation_data=dataset_fit_dev_hp, verbose =0, callbacks=[stop_early]) #callbacks=[PlotLossesCallback()],

 

    return out.history[val_metric][-1]#val_sparse_categorical_accuracy


L1L2_reg = ( 0.0001, 0.1)
drop_reg = ( 0.1, 0.5)

para = {
    "hp_units": (32,1024),
     "hp_enc_drop": drop_reg,
     "hp_dec_drop": drop_reg,
     "hp_enc_reg": L1L2_reg,
     "hp_dec_reg": L1L2_reg,
     "lr": (0.00001, 0.001)
     }

from functools import partial

fit_with_partial = partial(get_bayesian_tunning)

optimizer = BayesianOptimization(
    f = fit_with_partial,
    pbounds =para,
    verbose=2,  # verbose = 1 prints only when a maximum is observed, verbose = 0 is silent
    random_state=1
)
para


In [ ]:
aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa

In [ ]:
try:
  load_logs(optimizer, logs=["/content/drive/My Drive/Colab Notebooks/Translator/bayesian_logs.json"]);
except :
  print("File bayesian_logs.json does not exists")

In [ ]:
logger = JSONLogger(path="/content/drive/My Drive/Colab Notebooks/Translator/bayesian_logs.json")
optimizer.subscribe(Events.OPTIMIZATION_STEP, logger)

In [ ]:
optimizer.maximize(init_points=20, n_iter=30)

In [ ]:
for i, res in enumerate(optimizer.res):
    print("Iteration {}: \n\t{}".format(i, res))

print(optimizer.max)

# Performing Hyperparam tunning Using Talo  + TPU

In [ ]:

with tpu_strategy.scope():

  custom_translator = TrainCustomTranslator(
      emb_dim= emb_dim,
      units = 900,
      encoder_vocab_size= encoder_vocab_size,
      decoder_vocab_size= decoder_vocab_size,
      embeddings_dec_matrix = embeddings_eng_matrix,
      embeddings_enc_matrix = embeddings_pt_matrix,
      dec_tokenizer = eng_tokenizer,
      dec_sequence_size = en_sequence_size, 
      enc_sequence_size = pt_sequence_size,
      use_tf_function=True,
      enc_drop =0.5,
      enc_rec_drop = 0.5, 
      enc_reg = 0.001,
      dec_drop = 0.5, 
      dec_rec_drop = 0.5, 
      dec_reg = 0.001
      )

  lr = 0.0005
  # Configure the loss and optimizer
  custom_translator.compile(
      optimizer=tf.optimizers.Adam(lr),
      loss=MaskedLossCustom(),
      metrics = [tf.metrics.SparseCategoricalAccuracy()] #["tf.metrics.SparseCategoricalAccuracy()"] AccuracyCustom()
  )

  if TRAIN_MODEL:    

    history = custom_translator.fit(dataset_fit,  epochs=training_epochs, callbacks=[PlotLossesCallback()], steps_per_epoch = train_size,  shuffle= True,  validation_data = dataset_fit_dev, validation_steps=10)
    custom_translator.save_weights("custom_translator")
  else : 
      custom_translator.load_weights("custom_translator")


In [ ]:
from rouge import Rouge #https://towardsdatascience.com/introduction-to-text-summarization-with-rouge-scores-84140c64b471
ROUGE = Rouge()

def Minimum_Bayes_Risk(list_of_candidates, field = "rouge-l", candidate_reference = None):
    candidate_score = {}
    max_score = 0
    best_score_candidate= ''
    best_candidate_index = -1
    if candidate_reference == None :
        for candidate_ref in list_of_candidates:
            
            rest_of_candidates = copy.deepcopy(list_of_candidates)
            if candidate_ref in rest_of_candidates : rest_of_candidates.remove(candidate_ref)
            score = 0

            for candidate in rest_of_candidates:
                score += ROUGE.get_scores(candidate, candidate_ref)[0][field]["f"]
            
            score = score/len(rest_of_candidates)

            candidate_score[candidate_ref] = score

            if score >= max_score:
                best_score_candidate =candidate_ref
                best_candidate_index = list_of_candidates.index(candidate_ref)
                max_score = score

    else:
            
            for candidate in list_of_candidates:
                score = 0
                score = ROUGE.get_scores(candidate, candidate_reference)[0][field]["f"]
                if score >= max_score:
                    best_score_candidate =candidate


    return candidate_score, best_score_candidate, best_candidate_index


def plot_attention(attention, sentence, predicted_sentence):
  fig = plt.figure(figsize=(30,30))
  ax = fig.add_subplot(1, 1, 1)
  ax.matshow(attention, cmap='viridis')

  fontdict = {'fontsize': 14}

  ax.set_xticklabels([''] + sentence, fontdict=fontdict, rotation=90)
  ax.set_yticklabels([''] + predicted_sentence, fontdict=fontdict)

  ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
  ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

  plt.show()


In [ ]:
vocab_size = decoder_vocab_size
token_mask_ids = np.array([eng_tokenizer.word_index[x] for x in  [ '<oov>' ]])[:, None]#
token_mask = np.zeros([vocab_size], dtype=np.bool)
token_mask[np.array(token_mask_ids)] = True

print("token_mask", token_mask)

def translate_sequence(input_tokens, temperature=0.5, number_of_candidates = 10, loop= False):

  attention_per_candidate = []
  result_text_per_candidate = []
  input_text_per_candidate = []
  output_text_per_candidate = []
  

  for c in range(number_of_candidates):
    
    batch_size_tmp = len(input_tokens)

    last_dec = None
    
    max_length = input_tokens.shape[1] #np.count_nonzero(input_mask)
    
    enc_output,  enc_hidden, enc_cell = custom_translator.encoder(input_tokens)
    dec_state = ( enc_hidden, enc_cell)

    x = tf.expand_dims([eng_tokenizer.word_index['<sos>']]*batch_size_tmp , 1)
    result_tokens = []
    input_text=[]
    output_text=[]
    attention = np.zeros((pt_sequence_size, en_sequence_size))

    for t in range(pt_sequence_size):

      input_text.append(' '.join([pt_tokenizer.index_word[w.numpy()] for w in  input_tokens[0] if w != 0 ]))

      input_mask = x!= 0

      predictions, dec_state, attention_weights, last_dec = custom_translator.decoder(x, dec_state, enc_output, last_dec, input_mask)

      attention_weights = tf.reshape(attention_weights, (-1, ))
      attention[t] = attention_weights.numpy()

      predicted_logits  = predictions[:,-1,:]/temperature
      predicted_logits = tf.where(token_mask, -np.inf, predicted_logits)
      predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
      new_tokens = tf.squeeze(predicted_ids, axis=-1)

      if loop:
          x = new_tokens.numpy()
      else : 
          x = list(x[0])
          x = [i for i in x if i != 0]
          x.append(new_tokens.numpy()[0])

      output_text.append(' '.join([eng_tokenizer.index_word[w] for w in  x if w != 0 ]))
      x = tf.reshape(tf.convert_to_tensor(x), (len(input_tokens),len(x)))

      result_tokens.append(new_tokens.numpy()[0])

      if eng_tokenizer.word_index['<eos>'] == new_tokens.numpy()[0]:
          break
      
          
    result_text = ' '.join([eng_tokenizer.index_word[w] for w in  result_tokens if w != 0])

    
    attention_per_candidate.append(attention)
    result_text_per_candidate.append(result_text)
    input_text_per_candidate.append(input_text)
    output_text_per_candidate.append(output_text)

  candidate_score, best_score_candidate, best_candidate_index = Minimum_Bayes_Risk(result_text_per_candidate)

  attention_stack = attention_per_candidate[best_candidate_index]
  result_text = result_text_per_candidate[best_candidate_index]
  input_text = input_text_per_candidate[best_candidate_index]
  output_text = output_text_per_candidate[best_candidate_index]


  return {'pt_sentence': result_text, 'attention': attention_stack, "input_text" : input_text, "output_text" : output_text, "candidates": result_text_per_candidate}

In [ ]:
decoder_vocab_size

In [ ]:
print(' '.join([pt_tokenizer.index_word[i] for i in x[0] if i != 0])," ---> ",  ' '.join([eng_tokenizer.index_word[i] for i in y[0] if i != 0]))

sentences = ' '.join([pt_tokenizer.index_word[i] for i in x[0] if i != 0])
result = translate_sequence(input_tokens = tf.expand_dims(x[0],0), temperature=0.5, number_of_candidates = 2, loop= True)

attention_plot, res= result["attention"], result["pt_sentence"]
print(sentences," ---> ",res)

attention_plot = attention_plot[:len(res.split(' ')), :len(sentences.split(' '))]
plot_attention(attention_plot, sentences.split(' '), res.split(' '))



In [ ]:
"""
from nltk.translate.bleu_score import corpus_bleu


def word_for_id(integer, tokenizer):
    # map an integer to a word
    for word, index in tokenizer.word_index.items():
        if index == integer:
            return word
    return None

def predict_seq(model, tokenizer, source):
    # generate target from a source sequence
    prediction = model.predict(source, verbose=0)[0]
    integers = [np.argmax(vector) for vector in prediction]
    target = list()
    for i in integers:
        word = word_for_id(i, tokenizer)
        if word is None:
            break
        target.append(word)
    return ' '.join(target)

def bleu_score(model, tokenizer, sources, raw_dataset):
    # Get the bleu score of a model
    actual, predicted = [], []
    for i, source in enumerate(sources):
        # translate encoded source text
        source = source.reshape((1, source.shape[0]))
        translation = predict_seq(model, tokenizer, source)
        raw_target, raw_src = raw_dataset[i]
        actual.append([raw_target.split()])
        predicted.append(translation.split())
        
    bleu_dic = {}
    bleu_dic['1-grams'] = corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0))
    bleu_dic['1-2-grams'] = corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0))
    bleu_dic['1-3-grams'] = corpus_bleu(actual, predicted, weights=(0.3, 0.3, 0.3, 0))
    bleu_dic['1-4-grams'] = corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25))
    
    return bleu_dic

bleu_train = bleu_score(model, pt_tokenizer, x_dev, train)
bleu_test = bleu_score(model, pt_tokenizer, x_dev, test)

plt.figure()
plt.bar(x = bleu_train.keys(), height = bleu_train.values())
plt.title("BLEU Score with the training set")
plt.ylim((0,1))
plt.show()

plt.figure()
plt.bar(x = bleu_test.keys(), height = bleu_test.values())
plt.title("BLEU Score with the test set")
plt.ylim((0,1))
plt.show()"""

In [ ]:
for i, o in dataset_fit_dev.take(1):
    print()
k = 20 

print(' '.join([eng_tokenizer.index_word[i] for i in x[k] if i != 0])," ---> ",  ' '.join([pt_tokenizer.index_word[i] for i in y[k] if i != 0]))

sentences = ' '.join([eng_tokenizer.index_word[i] for i in x[k] if i != 0])
result = translate_sequence(input_tokens = tf.expand_dims(x[k],0), temperature=0.5, number_of_candidates = 2, loop= True)

attention_plot, res= result["attention"], result["pt_sentence"]
print(sentences," ---> ",res)

attention_plot = attention_plot[:len(res.split(' ')), :len(sentences.split(' '))]
plot_attention(attention_plot, sentences.split(' '), res.split(' '))